import [library] as [nickname]

from [library] import [function]

In [3]:
import pandas as pd

from matplotlib import pyplot as plt
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler 

Importing data from a CSV file using built in read_csv function from pandas (pd) library

In [4]:
train_data = pd.read_csv("https://raw.githubusercontent.com/MobiusYeeitus/Hateful-Meme-Detection/breakdown/files/df_c.csv") 
#path will definitely be changed to a github-repo/local later

Calling the object to visualize the object

In [5]:
train_data

,Unnamed: 0,text,identity
0,0,house dem aide didnt comeys letter jason chaff...,1
1,1,feeling life circles roundabout heads straight...,0
2,2,truth fired october 29 2016 tension intelligen...,1
3,3,videos 15 civilians killed single airstrike id...,1
4,4,print iranian woman sentenced years prison ira...,1
...,...,...,...
833,833,ceo trade association american fashion designe...,0
834,834,november 12 2016 348 ads craigslist reveal pai...,1
835,835,men exercise strenuously lower libido workouts...,0
836,836,humans free baking soda coconut oil kill cance...,1


Example of read_csv and to_csv functions

In [6]:
#example
from pandas import read_csv
example_data=read_csv("https://raw.githubusercontent.com/MobiusYeeitus/Hateful-Meme-Detection/breakdown/files/df.csv") #read from web
example_data.to_csv("D:/df.csv") #save to local path
example_data=read_csv("D:/df.csv") #read from local path
example_data.sample(10)

,Unnamed: 0.1,Unnamed: 0,id,title,author,text,label
315,315,315,315,Donald Trump to March for Life: ‘You Have My F...,Charlie Spiering,President Donald Trump signaled his support fo...,0
450,450,450,450,Voters Can Fight Back Against Election Fraud,NaN,Voters Can Fight Back Against Election Fraud N...,1
916,916,916,916,Former US Attorney for DC: New Hillary Email P...,Chris Menahan,Former United States Attorney for the District...,1
247,247,247,247,A Scaredy-Cat’s Investigation Into Why People ...,Steph Yin,Halloween is here again. That means your hav...,0
142,142,142,142,Gorafi Magazine : Entretien exclusif avec Bara...,NaN,NaN,1
439,439,439,439,"Gorsuch, London, Republican Party: Your Thursd...",Karen Zraick and Sandra Stevenson,(Want to get this briefing by email? Here’s th...,0
832,832,832,832,Susan Rice: U.S. Must Integrate LGBT Rights in...,Kaitlyn Stegall,"October 28, 2016 Susan Rice: U.S. Must Integra...",1
950,950,950,950,"NATIONAL REVIEW, Conservatism Inc., Plan To Ca...",James Kirkpatrick,,1
998,998,998,998,Saudi Arabia announces date certain for the ap...,NaN,"Email Saudi Arabia announces the VAT Act, whic...",1
390,390,390,390,"UConn’s Recipe for Success Is to Run, Run and ...",Jeré Longman,"STORRS, Conn. — After losing to Connecticut...",0


Creates a list called 'text' and appends all the words in the dataset into the list

In [7]:
text = []
for i in train_data['text']:
    text.append(i)

Creates a matrix representation of the list 'text' based on One-Hot Encoding/Representation

Topic to study -> One-Hot Representation

In [8]:
tf_transfomer = TfidfVectorizer(stop_words='english',decode_error='ignore')
X_train_counts_tf = tf_transfomer.fit_transform(text)
scaler = StandardScaler(with_mean=False)
x_train_counts_tf = scaler.fit_transform(X_train_counts_tf)

Displays shape of 'X_train_counts_tf' matrix

In [9]:
X_train_counts_tf.shape

(838, 39716)

This needs a redo, 10000 iterations ain't it chief, even Google Colab crashed as it ran out of RAM. They don't even save the values spat out by the process, whY?

In [84]:
train_cut = 8000
num_folds = 10
seed = 7
scoring = 'accuracy'
param_grid = {}
param_grid['C'] = [0.1,0.2,0.3,0.5,0.8,1,5,13,15]
param_grid['solver'] = ['newton-cg','lbfgs','liblinear','sag','saga']
model = LogisticRegression(max_iter = 10000)
kfold = KFold(n_splits=num_folds,random_state=seed,shuffle = True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X=X_train_counts_tf,y=train_data['text'])
print('Best : %s Using %s' % (grid_result.best_score_, grid_result.best_params_))
param_grid = {}
param_grid['alpha'] = [0.001,0.01,0.05,0.1,0.5,1,1.5,2]
model = MultinomialNB()
kfold = KFold(n_splits=num_folds,random_state=seed,shuffle = True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X=X_train_counts_tf[:train_cut],y=train_data['text'])
print('Best : %s Using %s' % (grid_result.best_score_, grid_result.best_params_))
param_grid = {}
param_grid['n_estimators'] = [10,50,100,150,200]
param_grid['max_features'] = ['auto','sqrt','log2']
param_grid['max_depth'] = [3,4,5,6,7,8,9,10]
model = RandomForestClassifier()
kfold = KFold(n_splits=num_folds,random_state=seed,shuffle = True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X=X_train_counts_tf[:train_cut],y=train_data['text'])
print('Best : %s Using %s' % (grid_result.best_score_, grid_result.best_params_))
#SVM
param_grid = {'C':[0.1,0.2,0.3,0.5,0.8,1,5,13,15,20]}
model = SVC(max_iter =100000)
kfold = KFold(n_splits=num_folds,random_state=seed,shuffle = True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X=X_train_counts_tf,y=train_data['text'])
print('Best : %s Using %s' % (grid_result.best_score_, grid_result.best_params_))
#KNN
param_grid = {}
param_grid['n_neighbors'] = [3,5,10,20,30]
param_grid['weights'] = ['uniform','distance']
model = KNeighborsClassifier()
kfold = KFold(n_splits=num_folds,random_state=seed,shuffle = True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X=X_train_counts_tf[:train_cut],y=train_data['text'])
print('Best : %s Using %s' % (grid_result.best_score_, grid_result.best_params_))